In [10]:
import warnings
warnings.filterwarnings('ignore')

import warnings
import pandas as pd
import geopandas as gpd
from pathlib import Path

# Suprimir advertencias
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


def load_geodata(file_path):
    """Carga datos geoespaciales desde un archivo."""
    try:
        return gpd.read_file(file_path)
    except Exception as e:
        raise FileNotFoundError(f"Error al leer el archivo {file_path}: {e}")


def get_user_input():
    """Solicita entrada del usuario para provincia y municipio."""
    provincia = input("Indica el nombre de la provincia (ej: Madrid, Barcelona, Valencia): ").strip()
    municipio = input("Indica el nombre del municipio (ej: Madrid, Barcelona, Valencia): ").strip()
    return provincia, municipio


def filter_provincia(provincias_gdf, provincia_nome):
    """Filtra y elimina la provincia seleccionada de los datos."""
    provincia_normalizada = provincia_nome.lower()
    provincia_encontrada = provincias_gdf[
        provincias_gdf["name"].str.strip().str.lower() == provincia_normalizada
    ]

    if provincia_encontrada.empty:
        raise ValueError(f"Provincia '{provincia_nome}' no encontrada.")

    otras_provincias = provincias_gdf[
        provincias_gdf["name"].str.strip().str.lower() != provincia_normalizada
    ]

    return otras_provincias


def process_municipios(municipios_gdf, municipio_nome, provincia_nome):
    """Procesa los municipios de la provincia seleccionada, separando el municipio de interés."""
    provincia_normalizada = provincia_nome.lower()
    municipio_normalizado = municipio_nome.lower()

    # Filtra solo los municipios de la provincia indicada
    municipios_da_provincia = municipios_gdf[
        municipios_gdf["prov_name"].str.strip().str.lower() == provincia_normalizada
    ]

    if municipios_da_provincia.empty:
        raise ValueError(f"Ningún municipio encontrado para la provincia '{provincia_nome}'.")

    # Verifica si el municipio existe en la provincia
    municipio_selecionado = municipios_da_provincia[
        municipios_da_provincia["name"].str.strip().str.lower() == municipio_normalizado
    ]

    if municipio_selecionado.empty:
        raise ValueError(f"Municipio '{municipio_nome}' no encontrado en la provincia '{provincia_nome}'.")

    # Configura el municipio seleccionado
    municipio_selecionado = municipio_selecionado.copy()
    # Elimina el prefijo MUN- si existe, antes de añadir AOI-
    id_limpo = municipio_selecionado["id"].astype(str).str.replace("MUN-", "", regex=False)
    municipio_selecionado["id"] = "AOI-" + id_limpo
    municipio_selecionado["poly_type"] = "core"

    # Otros municipios de la misma provincia (excepto el seleccionado)
    outros_municipios_provincia = municipios_da_provincia[
        municipios_da_provincia["name"].str.strip().str.lower() != municipio_normalizado
    ]

    return municipio_selecionado, outros_municipios_provincia


def create_project(outras_provincias, outros_municipios_provincia, municipio_selecionado):
    """Crea el proyecto final concatenando todas las capas."""
    return pd.concat([outras_provincias, outros_municipios_provincia, municipio_selecionado],
                    ignore_index=True)


def save_project(projeto_gdf, municipio_nome):
    """Guarda el proyecto en un archivo GeoJSON."""
    output_filename = f"{municipio_nome.replace(' ', '_')}_AOI.geojson"
    try:
        projeto_gdf.to_file(output_filename, driver='GeoJSON')
        print(f"Proyecto guardado con éxito en {output_filename}")
    except Exception as e:
        raise IOError(f"Error al guardar el proyecto: {e}")


def main():
    """Función principal del programa."""
    try:
        provincia, municipio = get_user_input()

        provincias = load_geodata("esp_provinces_SIMP.geojson")
        municipios = load_geodata("esp_municipalities_SIMP.geojson")

        outras_provincias = filter_provincia(provincias, provincia)
        municipio_selecionado, outros_municipios_provincia = process_municipios(municipios, municipio, provincia)

        projeto = create_project(outras_provincias, outros_municipios_provincia, municipio_selecionado)
        save_project(projeto, municipio)

    except (FileNotFoundError, ValueError, IOError) as e:
        print(f"Error: {e}")
    except KeyboardInterrupt:
        print("\nOperación cancelada por el usuario.")
    except Exception as e:
        print(f"Error inesperado: {e}")


if __name__ == "__main__":
    main()


Indica el nombre de la provincia (ej: Madrid, Barcelona, Valencia):  Illes Balears
Indica el nombre del municipio (ej: Madrid, Barcelona, Valencia):  Palma


Proyecto guardado con éxito en Palma_AOI.geojson


In [11]:
teste = gpd.read_file("Palma_AOI.geojson")

In [12]:
teste

,id,name,poly_type,prov_name,geometry
0,PRO-52,Melilla,periphery,None,"POLYGON ((-2.95264 35.3203, -2.95533 35.31854,..."
1,PRO-27,Lugo,periphery,None,"MULTIPOLYGON (((-7.93075 42.5786, -7.928 42.57..."
2,PRO-34,Palencia,periphery,None,"MULTIPOLYGON (((-5.00214 42.28913, -5.00488 42..."
3,PRO-13,Ciudad Real,periphery,None,"MULTIPOLYGON (((-4.26847 38.34703, -4.2693 38...."
4,PRO-35,Las Palmas,periphery,None,"MULTIPOLYGON (((-15.37997 27.9313, -15.37815 2..."
...,...,...,...,...,...
115,MUN-07037,Es Mercadal,periphery,Illes Balears,"MULTIPOLYGON (((4.07321 40.09204, 4.0762 40.09..."
116,MUN-07010,Bunyola,periphery,Illes Balears,"POLYGON ((2.65911 39.6584, 2.66061 39.65737, 2..."
117,MUN-07061,Sóller,periphery,Illes Balears,"MULTIPOLYGON (((2.72781 39.82339, 2.72632 39.8..."
118,MUN-07033,Manacor,periphery,Illes Balears,"POLYGON ((3.15322 39.57578, 3.15189 39.57457, ..."


In [2]:
provincias = gpd.read_file("esp_provinces_SIMP.geojson")

In [5]:
muns = gpd.read_file("esp_municipalities_SIMP.geojson")

In [9]:
muns[muns["prov_name"] == 'Illes Balears']['name'].unique()

array(['Sant Joan', 'Selva', 'Capdepera', 'Sencelles', 'Formentera',
       'Maria de la Salut', 'Costitx', 'Sant Antoni de Portmany',
       'Sant Llorenç des Cardassar', 'Vilafranca de Bonany', 'Santanyí',
       'Petra', 'Santa Eulària des Riu', 'Eivissa', 'es Castell',
       'Consell', 'Lloseta', 'Sant Josep de sa Talaia', 'Algaida',
       'Felanitx', 'Búger', 'Montuïri', 'Binissalem', 'Son Servera',
       'Calvià', 'Escorca', 'Maó', 'Mancor de la Vall',
       'Sant Joan de Labritja', 'Llubí', 'Deià', 'Sa Pobla', 'Fornalutx',
       'Lloret de Vistalegre', 'Es Migjorn Gran', 'Artà', 'Marratxí',
       'Sant Lluís', 'Santa María del Camí', 'Maó-Mahón', 'Llucmajor',
       'ses Salines', 'Campos', 'Porreres', 'Santa Eugènia', 'Alaior',
       'Pollença', 'Banyalbufar', 'Muro', 'Alcúdia', 'Esporles',
       'Ferreries', 'Andratx', 'Sineu', 'Ciutadella de Menorca', 'Alaró',
       'Ariany', 'Inca', 'Santa Margalida', 'Estellencs', 'Palma',
       'Valldemossa', 'Puigpunyent', 'Camp